In [1]:
import sys; sys.path.append("../../wwa"); from wwa import *

mapproj = cartopy.crs.PlateCarree()

fpath = "/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/FS_heat/"

Xn,Xx,Yn,Yx = [0,35,55,75]
xn,xx,yn,yx = [4,30,60,72]

# Shapefile for study region

In [16]:
sf = regionmask.defined_regions.natural_earth_v5_0_0.countries_50.to_geodataframe().set_index("names").loc[["Sweden", "Norway", "Finland"]]
sf = main_polygon(sf.dissolve())
sf.to_file("sf_fennoscandia")

# Get gridded data

## ERA5

In [ ]:
era5_fpath = fpath+"era5_tmax_daily_eu_extended_0-35E_55-75N_-999-lastyear.nc"
! wget https://climexp.knmi.nl/data/era5_tmax_daily_eu_extended_0-35E_55-75N_-999-lastyear_su.nc -O $era5_fpath

era5_fpath = fpath+"era5_tmin_daily_eu_extended_0-35E_55-75N_-999-lastyear.nc"
! wget https://climexp.knmi.nl/data/era5_tmin_daily_eu_extended_0-35E_55-75N_-999-lastyear_su.nc -O $era5_fpath

In [20]:
varnm = "tmin"

lsm = xr.open_dataset(fpath+"era5_000000_lsm_eu.nc").lsm.squeeze(drop = True).rename(latitude = "lat", longitude = "lon")

da = xr.open_dataset(fpath+"era5_"+varnm+"_daily_eu_extended_0-35E_55-75N_-999-lastyear.nc")[varnm]
da = da.where(lsm.sel(lon = da.lon, lat = da.lat) >= 0.5, drop = True)
da.to_netcdf(fpath+"era5_"+varnm+"_daily_land_0-35E_55-75N.nc")

## E-Obs

In [ ]:
# lower-res data from Climate Explorer
! cd $fpath; wget https://climexp.knmi.nl/data/tx_0.25deg_reg_v31.0eu_0-35E_55-71.88N_firstyear-lastyear.nc
! cd $fpath; wget https://climexp.knmi.nl/data/tn_0.25deg_reg_v31.0eu_0-35E_55-71.88N_-999-lastyear.nc

### Retrieve latest from Copernicus (both resolutions for now, in case of gaps)

In [ ]:
! cd $fpath; wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/Grid_0.1deg_reg_ensemble/tx_ens_mean_0.1deg_reg_v31.0e.nc
! cd $fpath; wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/Grid_0.1deg_reg_ensemble/tn_ens_mean_0.1deg_reg_v31.0e.nc

In [ ]:
# historic data
! cd $fpath; wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/Grid_0.25deg_reg_ensemble/tx_ens_mean_0.25deg_reg_v31.0e.nc
! cd $fpath; wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/Grid_0.25deg_reg_ensemble/tn_ens_mean_0.25deg_reg_v31.0e.nc

In [ ]:
! cd $fpath; wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/months/ens/tx_0.25deg_day_2025_grid_ensmean.nc
! cd $fpath; wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/months/ens/tn_0.25deg_day_2025_grid_ensmean.nc

### Extract wider region

In [2]:
varnm = "tn"
res = "0.25"

In [ ]:
ds_2025 = xr.open_dataset(fpath+varnm+"_"+res+"deg_day_2025_grid_ensmean.nc").dropna("time", "all").sel(longitude = slice(Xn,Xx), latitude = slice(Yn,Yx))

In [ ]:
ds_hist = xr.open_dataset(fpath+varnm+"_ens_mean_"+res+"deg_reg_v31.0e.nc").dropna("time", "all").sel(longitude = slice(Xn,Xx), latitude = slice(Yn,Yx))

In [ ]:
ds = xr.merge([ds_hist, ds_2025], "time")

In [7]:
ds = xr.open_mfdataset([fnm for fnm in glob.glob(fpath+varnm+"*"+res+"*.nc") if not "lastyear" in fnm])
ds = ds[list(ds.data_vars)[0]].sel(longitude = slice(Xn,Xx), latitude = slice(Yn,Yx))
ds = ds.dropna("time", "all")

/rds/general/user/cb2714/home/anaconda3/envs/wwa/lib/python3.10/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [ ]:
ds.to_netcdf(fpath+"eobs_"+varnm+"_"+res+"deg.nc")

TypeError: unsupported type for timedelta days component: numpy.int64

## CPC

In [ ]:
! cd $fpath; wget https://climexp.knmi.nl/data/tmin_cpc_daily_0-35E_55-75N_-999-lastyear_su.nc
! cd $fpath; wget https://climexp.knmi.nl/data/tmax_cpc_daily_0-35E_55-75N_-999-lastyear_su.nc

## MSWX

In [2]:
outpath = "/rds/general/user/cb2714/home/99_ephemera/mswx_fs/"

In [ ]:
for y in range(2025,1978,-1):

    print(y)
    new_fnm = outpath+"tmax_mswx_"+str(y)+".nc"
    if os.path.exists(new_fnm): continue
        
    ds = xr.open_mfdataset("/rds/general/user/cb2714/home/00_WWA_project_folder/live/data/mswx/Tmax/"+str(y)+"/*.nc")
    ds = ds.sel(lon = slice(Xn,Xx), lat = slice(Yx,Yn)).rename(air_temperature = "tmax")
    ds.to_netcdf(new_fnm)

In [23]:
# compile & save
wrap_lon(xr.open_mfdataset(outpath+"*.nc")).to_netcdf("data/tmax_mswx.nc")

# Time series over study region

In [34]:
sf = gpd.read_file("sf_fennoscandia")

ndays = 14

tx_list = {"era5" : "era5_tmax_daily_land_0-35E_55-75N.nc",
           "eobs" : "tx_0.25deg_reg_v31.0eu_0-35E_55-71.88N_firstyear-lastyear.nc",
           "cpc" : "tmax_cpc_daily_0-35E_55-75N_-999-lastyear_su.nc"}

tn_list = {"era5" : "era5_tmin_daily_land_0-35E_55-75N.nc",
           "eobs" : "tn_0.25deg_reg_v31.0eu_0-35E_55-71.88N_-999-lastyear.nc",
           "cpc" : "tmin_cpc_daily_0-35E_55-75N_-999-lastyear_su.nc"}

In [35]:
for varnm in ["tx", "tn"]:
    vnm = {"tx" : "max", "tn" : "min"}[varnm]

    ds_list = eval(varnm+"_list")
    
    for ds_nm, fnm in ds_list.items():

        print(fnm)
    
        da = wrap_lon(xr.open_dataset(fpath+fnm))
        da = da[list(da.data_vars)[0]]
        
        if "longitude" in da.dims:
            da = da.rename(longitude = "lon", latitude = "lat")

        rm = regionmask.mask_geopandas(sf, da)

        ts = da.where(rm == 0).mean(["lat", "lon"])
        ts.to_dataframe().to_csv("data/"+varnm+"-fs-daily_"+ds_nm+".csv")

        if ds_nm == "eobs":
            ts = ts.sel(time = slice(None, "2024"))
            
        # create annual time series for Climate Explorer
        txnx = ts.rolling(time = ndays, center = False).mean().resample(time = "AS").max()
        df = txnx.assign_coords(time = txnx.time.dt.year).rename(time = "year").to_dataframe()

        csv_fnm = "fs-heatwave_"+varnm+str(ndays)+"x_"+ds_nm
        csv_fullpath = "ts/"+csv_fnm+".dat"
        if len(csv_fnm) > 61: print("! Filename too long: ", csv_fnm)
        
        # create header lines for upload to Climate Explorer 
        str1 = "# contact :: "+ds_nm.upper()+" "+varnm+str(ndays)+"x over Norway, Sweden and Finland - Fennoscandia heatwave, c.barnes22@imperial.ac.uk"
        str2 = "# "+varnm+str(ndays)+"x [degC] - annual maximum of "+str(ndays)+"-day average of daily "+vnm+"imum temperatures averaged over Norway, Sweden and Finland at "+csv_fnm+".dat"
        head = "# year "+varnm+str(ndays)+"x"

        # make .dat file
        ! echo "$str1 " > $csv_fullpath
        ! echo "$str2" >> $csv_fullpath
        ! echo "$head" >> $csv_fullpath
        df.to_csv(csv_fullpath, sep = " ", mode = "a", header = False)

era5_tmax_daily_land_0-35E_55-75N.nc
tx_0.25deg_reg_v31.0eu_0-35E_55-71.88N_firstyear-lastyear.nc
tmax_cpc_daily_0-35E_55-75N_-999-lastyear_su.nc
era5_tmin_daily_land_0-35E_55-75N.nc
tn_0.25deg_reg_v31.0eu_0-35E_55-71.88N_-999-lastyear.nc
tmin_cpc_daily_0-35E_55-75N_-999-lastyear_su.nc
